# Лабораторная работа 4. Обучение с учителем. Задача классификации. Предобработка

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Задание 2.1

- Загрузите датасет для классификации (см. файл Excel для Лаб.работы №3), проведите предобработку

## Предобработка

In [2]:
df = pd.read_csv('../../../Datasets/Lab4_DataSet4.csv')

In [3]:
df.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,400740,XE,3036,IAH,CHS,4.0,1195,131.0,0
1,194808,UA,315,DEN,ONT,7.0,707,145.0,0
2,526903,OO,6673,SFO,MRY,5.0,499,42.0,1
3,477937,WN,3668,PHX,ONT,2.0,810,70.0,1
4,48180,AS,623,LAS,PDX,5.0,985,143.0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           400000 non-null  int64  
 1   Airline      399264 non-null  object 
 2   Flight       400000 non-null  int64  
 3   AirportFrom  400000 non-null  object 
 4   AirportTo    398843 non-null  object 
 5   DayOfWeek    399673 non-null  float64
 6   Time         400000 non-null  int64  
 7   Length       399517 non-null  float64
 8   Delay        400000 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 27.5+ MB


In [5]:
# Функция, которая из словаря делает запрос к таблице Pandas, притом удаляет запросы с nan
def my_query(dict_query: dict) -> str:
    for x in dict_query:
        obj_type = type(dict_query[x])
        if obj_type == str:
            dict_query[x] = '"'+dict_query[x]+'"'
        else:
            dict_query[x] = str(dict_query[x])
    return ' & '.join([x + ' == ' + dict_query[x] for x in dict_query if dict_query[x] != 'nan'])

Пример: Flight == 1092 & AirportFrom == "TPA" & AirportTo == "DTW" & Length == 165.0

In [6]:
# строки с нулевым значением столбца 'Airlines'
z = df[df['Airline'].isna()]

airlines = [df.query(
        my_query(
            {'Flight': df.iloc[x]['Flight'],
            'AirportFrom': df.iloc[x]['AirportFrom'],
            'AirportTo': df.iloc[x]['AirportTo'],
            'Length': df.iloc[x]['Length']
            }
        )
    )['Airline'].mode() for x in z.index
]

airlines = pd.Series(airlines, index=z.index).apply(lambda x: x.iat[0] if not x.empty else np.nan)

df['Airline'] = df['Airline'].fillna(airlines)

In [7]:
# строки с нулевым значением столбца 'AirportTo'
z = df[df['AirportTo'].isna()]

AirportTo = [df.query(
        my_query(
            {'Airline': df.iloc[x]['Airline'],
             'Flight': df.iloc[x]['Flight'],
            'AirportFrom': df.iloc[x]['AirportFrom'],
            }
        )
    )['AirportTo'].mode() for x in z.index
]

AirportTo = pd.Series(AirportTo, index=z.index).apply(lambda x: x.iat[0] if not x.empty else np.nan)

df['AirportTo'] = df['AirportTo'].fillna(AirportTo)

In [8]:
# строки с нулевым значением столбца 'AirportTo'
z = df[df['DayOfWeek'].isna()]

DayOfWeek = [df.query(
        my_query(
            {'Airline': df.iloc[x]['Airline'],
             'Flight': df.iloc[x]['Flight'],
            'AirportFrom': df.iloc[x]['AirportFrom'],
            'AirportTo': df.iloc[x]['AirportTo']
            }
        )
    )['DayOfWeek'].mode() for x in z.index
]

DayOfWeek = pd.Series(DayOfWeek, index=z.index).apply(lambda x: x.iat[0] if not x.empty else np.nan)

df['DayOfWeek'] = df['DayOfWeek'].fillna(DayOfWeek)

In [9]:
# строки с нулевым значением столбца 'Length'
z = df[df['Length'].isna()]

Length = [df.query(
        my_query(
            {'Airline': df.iloc[x]['Airline'],
             'Flight': df.iloc[x]['Flight'],
            'AirportFrom': df.iloc[x]['AirportFrom'],
            'AirportTo': df.iloc[x]['AirportTo']
            }
        )
    )['Length'].mode() for x in z.index
]

Length = pd.Series(Length, index=z.index).apply(lambda x: x.iat[0] if not x.empty else np.nan)

df['Length'] = df['Length'].fillna(Length)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           400000 non-null  int64  
 1   Airline      399984 non-null  object 
 2   Flight       400000 non-null  int64  
 3   AirportFrom  400000 non-null  object 
 4   AirportTo    399980 non-null  object 
 5   DayOfWeek    399992 non-null  float64
 6   Time         400000 non-null  int64  
 7   Length       399985 non-null  float64
 8   Delay        400000 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 27.5+ MB


In [11]:
df.isna().sum()

id              0
Airline        16
Flight          0
AirportFrom     0
AirportTo      20
DayOfWeek       8
Time            0
Length         15
Delay           0
dtype: int64

In [12]:
df = df.dropna()
df.isna().sum()

id             0
Airline        0
Flight         0
AirportFrom    0
AirportTo      0
DayOfWeek      0
Time           0
Length         0
Delay          0
dtype: int64

In [13]:
df.shape

(399941, 9)

In [14]:
df

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,400740,XE,3036,IAH,CHS,4.0,1195,131.0,0
1,194808,UA,315,DEN,ONT,7.0,707,145.0,0
2,526903,OO,6673,SFO,MRY,5.0,499,42.0,1
3,477937,WN,3668,PHX,ONT,2.0,810,70.0,1
4,48180,AS,623,LAS,PDX,5.0,985,143.0,0
...,...,...,...,...,...,...,...,...,...
399995,488611,AA,1533,ATL,DFW,3.0,415,150.0,0
399996,188847,US,486,RNO,PHX,7.0,365,107.0,0
399997,437638,OO,3111,CAK,MKE,7.0,640,84.0,0
399998,98020,AA,623,BWI,DFW,1.0,1000,220.0,0


In [23]:
df['DayOfWeek'] = df['DayOfWeek'].astype(int)
df['Length'] = df['Length'].astype(int)

In [16]:
from category_encoders import BinaryEncoder
bin_enc = BinaryEncoder()
df = bin_enc.fit_transform(df)

In [17]:
df.shape

(399941, 29)

In [18]:
df.head()

,id,Airline_0,Airline_1,Airline_2,Airline_3,Airline_4,Flight,AirportFrom_0,AirportFrom_1,AirportFrom_2,...,AirportTo_3,AirportTo_4,AirportTo_5,AirportTo_6,AirportTo_7,AirportTo_8,DayOfWeek,Time,Length,Delay
0,400740,0,0,0,0,1,3036,0,0,0,...,0,0,0,0,0,1,4.0,1195,131.0,0
1,194808,0,0,0,1,0,315,0,0,0,...,0,0,0,0,1,0,7.0,707,145.0,0
2,526903,0,0,0,1,1,6673,0,0,0,...,0,0,0,0,1,1,5.0,499,42.0,1
3,477937,0,0,1,0,0,3668,0,0,0,...,0,0,0,0,1,0,2.0,810,70.0,1
4,48180,0,0,1,0,1,623,0,0,0,...,0,0,0,1,0,0,5.0,985,143.0,0


In [19]:
import numpy as np
def reduce_mem_usage(df:pd.DataFrame):
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
    col_type = df[col].dtypes
    if str(col_type)[:5] == 'float':
      c_min = df[col].min()
      c_max = df[col].max()
      if c_min>np.finfo('f2').min and c_max < np.finfo('f2').max:
        df[col]=df[col].astype(np.float16)
      elif c_min>np.finfo('f4').min and c_max < np.finfo('f4').max:
        df[col]=df[col].astype(np.float32)
      else:
        df[col]=df[col].astype(np.float64)
    elif str(col_type)[:3] == 'int':
      c_min = df[col].min()
      c_max = df[col].max()
      if c_min>np.iinfo('i1').min and c_max < np.iinfo('i1').max:
        df[col]=df[col].astype(np.int8)
      elif c_min>np.iinfo('i2').min and c_max < np.iinfo('i2').max:
        df[col]=df[col].astype(np.int16)
      elif c_min>np.iinfo('i4').min and c_max < np.iinfo('i4').max:
        df[col]=df[col].astype(np.int32)
      elif c_min>np.iinfo('i8').min and c_max < np.iinfo('i8').max:
        df[col]=df[col].astype(np.int64)
    elif col == 'timestamp':
      df[col]=pd.to_datetime(df[col])
    elif str(col_type)[:8] != 'datetime':
      df[col]=df[col].astype('category')
  end_mem=df.memory_usage().sum() /1024**2
  print('Потребление памяти на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
  return df

In [24]:
df = reduce_mem_usage(df)

Потребление памяти на 1.91 Мб (минус 10.4 %)


In [21]:
df.to_csv('../../../Datasets/Lab4_DataSet4_Preprocessed.csv',index=False)